In [1]:
import pandas as pd
import datetime

In [2]:
#Colunas a serem lidas do dataset

date_col="date"
weight_col="hive_weight"
temp_col="hive_temperature"
humidity_col="hive_humidity"
amb_temp_col="ambient_temperature"
amb_humi_col="ambient_humidity"
cols_to_read=[date_col,weight_col, temp_col, humidity_col, amb_temp_col, amb_humi_col]

data_file="Arnas/arnas"
#data_file="Emil/emil"

df=pd.read_csv(filepath_or_buffer=data_file+".csv", usecols=cols_to_read)

In [3]:
#Funções que fazem a conversão da data

def date_to_nth_day(date, format='%Y-%m-%d %H:%M:%S'):
    date = datetime.datetime.strptime(date, format)
    new_year_day = pd.Timestamp(year=date.year, month=1, day=1)
    return (date - new_year_day).days + 1

def date_to_nth_week(date, format='%Y-%m-%d %H:%M:%S'):
    date = datetime.datetime.strptime(date, format)
    return (date.isocalendar()[1])

def date_to_weekday(date, format='%Y-%m-%d %H:%M:%S'):
    date = datetime.datetime.strptime(date, format)
    return (date.isocalendar()[2])

In [4]:
#Colunas a serem adicionadas

day_year_col=[]
week_year_col=[]
day_week_col=[]

for date in df.date:
    day_of_the_year = date_to_nth_day(date, format='%Y-%m-%d %H:%M:%S')
    day_year_col.append(day_of_the_year)
    week_of_the_year = date_to_nth_week(date, format='%Y-%m-%d %H:%M:%S')
    week_year_col.append(week_of_the_year)
    weekday = date_to_weekday(date, format='%Y-%m-%d %H:%M:%S')
    day_week_col.append(weekday)

In [5]:
#Adicionando as colunas ao dataset
df=df.assign(day_of_the_year=day_year_col,week_of_the_year=week_year_col,day_of_the_week=day_week_col)

In [6]:
#Salvando o dataset
df.to_csv(data_file+'_pre-processed-ts.csv', index = False)

PermissionError: [Errno 13] Permission denied: 'Arnas/arnas_pre-processed-ts.csv'

In [7]:
def DTWDistance(s1,s2,w=None):
        '''
        Calculates dynamic time warping Euclidean distance between two
        sequences. Option to enforce locality constraint for window w.
        '''
        DTW={}
        
        if w:
            w = max(w, abs(len(s1)-len(s2)))
    
            for i in range(-1,len(s1)):
                for j in range(-1,len(s2)):
                    DTW[(i, j)] = float('inf')
            
        else:
            for i in range(len(s1)):
                DTW[(i, -1)] = float('inf')
            for i in range(len(s2)):
                DTW[(-1, i)] = float('inf')
        
        DTW[(-1, -1)] = 0
        
        for i in range(len(s1)):
            if w:
                for j in range(max(0, i-w), min(len(s2), i+w)):
                    dist= (s1[i]-s2[j])**2
                    DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
            else:
                for j in range(len(s2)):
                    dist= (s1[i]-s2[j])**2
                    DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
            
        return (DTW[len(s1)-1, len(s2)-1]) 